# DOCUMENTATION FOR GNIMBUS v1

In [1]:
# TODO: sort imports

import numpy as np
import desdeo
from desdeo.problem import (PolarsEvaluator, Problem, VariableType,
                            variable_dict_to_numpy_array, Constraint,
                            ConstraintTypeEnum, Variable, VariableTypeEnum, ScalarizationFunction
                            )
from desdeo.tools.utils import (
    get_corrected_ideal_and_nadir,
    get_corrected_reference_point,
)
from desdeo.tools.scalarization import objective_dict_has_all_symbols

from desdeo.problem.testproblems import dtlz2, nimbus_test_problem

from gnimbus import (explain, voting_procedure, infer_classifications, agg_cardinal, infer_ordinal_classifications,
                     solve_intermediate_solutions, solve_sub_problems, convert_to_nimbus_classification, add_group_nimbusv2_sf_diff,
                     list_of_rps_to_dict_of_rps, dict_of_rps_to_list_of_rps)

from desdeo.problem import dtlz2, nimbus_test_problem, zdt1, zdt2
from desdeo.tools import IpoptOptions, PyomoIpoptSolver, add_asf_diff

from aggregate_classifications import aggregate_classifications


from desdeo.tools import (
    BaseSolver,
    SolverOptions,
    SolverResults,
    add_group_asf_diff,
    add_group_asf,
    add_group_guess_sf_diff,
    add_group_guess_sf,
    add_group_nimbus_sf_diff,
    add_group_nimbus_sf,
    add_group_stom_sf_diff,
    add_group_stom_sf,
    guess_best_solver,
    add_asf_diff,
    ScalarizationError,
    add_nimbus_sf_diff, add_nimbus_sf_nondiff
)
from desdeo.mcdm.nimbus import (
    generate_starting_point,
    infer_classifications,
    NimbusError
)

from aggregate_classifications import aggregate_classifications

## Setting up the problem

Set up the problem normally and use nimbus generate_starting_point.

In [2]:
n_variables = 8
n_objectives = 3
problem = dtlz2(n_variables, n_objectives)
solver_options = IpoptOptions()
# get some initial solution
initial_rp = {
    "f_1": 0.4, "f_2": 0.5, "f_3": 0.8
}
initial_result = generate_starting_point(problem, initial_rp)
initial_fs = initial_result.optimal_objectives
initial_fs

/home/jp/tyot/mop/desdeo/DESDEO/.venv/lib/python3.12/site-packages/pydantic/main.py:421: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `int` with value `5` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


{'f_1': 0.38509817535988233,
 'f_2': 0.48509817753491596,
 'f_3': 0.7850981807944739}

## Learning phase

### The solution process starts with the learning phase. 

We iterate until the DMs wish to go to the decision phase or based on a "stopping" criterion. For example, let us run 3 iterations.

In [3]:
# for first iteration
next_current_solution = initial_result.optimal_objectives
print(f"initial solution: {next_current_solution}")

initial solution: {'f_1': 0.38509817535988233, 'f_2': 0.48509817753491596, 'f_3': 0.7850981807944739}


## ITERATION STARTS HERE

#### PREFERENCES

In [4]:
dms_rps = {
    "DM1": {"f_1": 0.0, "f_2": next_current_solution["f_2"], "f_3": 1},  # improve f_1, keep f_2 same, impair f_3
    "DM2": {"f_1": 0.3, "f_2": 1, "f_3": 0.5},  # improve f_1 to 0.3, impair f_2, improve f_3 to 0.5
    "DM3": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.0},  # impair f_1 to 0.5, impair f_2 to 0.6, improve f_3
}

### Run iteration

GNIMBUS solve_sub_problems solves the MOP by using different scalarization functions. Currently, the list involves group_nimbus, group_stom, group_asf and group_guess. Adding more scalarization functions to this can be done easily.

In [5]:
num_desired = 4
solutions = solve_sub_problems(
    problem, next_current_solution, dms_rps, num_desired, decision_phase=False, create_solver=PyomoIpoptSolver, solver_options=solver_options
)
for s in solutions:
    print(f"Solution: {s.optimal_objectives}")

gnimbus = solutions[0].optimal_objectives
gstom = solutions[1].optimal_objectives
gasf = solutions[2].optimal_objectives
gguess = solutions[3].optimal_objectives

RPS {'f_1': 0.0, 'f_2': 0.48509817753491596, 'f_3': 1}
RPS {'f_1': 0.3, 'f_2': 1, 'f_3': 0.5}
RPS {'f_1': 0.5, 'f_2': 0.6, 'f_3': 0.0}


/home/jp/tyot/mop/desdeo/DESDEO/.venv/lib/python3.12/site-packages/pydantic/main.py:421: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `int` with value `5` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


Solution: {'f_1': 0.3850981774079426, 'f_2': 0.4850981843785722, 'f_3': 0.7850981755613075}
Solution: {'f_1': -1.505131080309009e-08, 'f_2': 0.9999999999999998, 'f_3': -1.161604257791914e-08}
Solution: {'f_1': 0.36843261262452104, 'f_2': 0.8535307961578433, 'f_3': 0.3684326125152709}
Solution: {'f_1': 0.9999999999999998, 'f_2': -1.3202070904492745e-08, 'f_3': -1.3202067299885665e-08}


### Select the next current iteration point by voting procedure

Voting procedure combines existing voting rules tried in the following order:
- Majority rule
- plurality rule
- if two solutions with most votes: we find intermediate solution
- if none above applies, we select the group_nimbus solution.

In [6]:
votes_idxs = {
        "DM1": 1,
        "DM2": 2,
        "DM3": 2,
}
voting_res = voting_procedure(problem, solutions, votes_idxs)
next_current_solution = voting_res.optimal_objectives
print("next current solution:", next_current_solution)

Majority winner 2
next current solution: {'f_1': 0.36843261262452104, 'f_2': 0.8535307961578433, 'f_3': 0.3684326125152709}


### Go to next iteration or move to decision phase

# "Decision phase"

In [7]:
# for other iterations
dms_rps = {
    "DM1": {"f_1": 0.3, "f_2": 0.6, "f_3": 0.6},
    "DM2": {"f_1": 0.3, "f_2": 0.2, "f_3": 0.5},
    "DM3": {"f_1": 0.4, "f_2": 0.1, "f_3": 0.4},
}

### Let us only propose one solution found by group nimbus which respects each DMs bounds.

We set decision phase = True to only use group_nimbus. Currently the num desired does not matter, but later may be useful.

In [8]:
num_desired = 1 
solutions = solve_sub_problems(
    problem, next_current_solution, dms_rps, num_desired, decision_phase=True, create_solver=PyomoIpoptSolver, solver_options=solver_options
)
gnimbus_solution = solutions[0].optimal_objectives
print("Final solution candidate:", gnimbus_solution)

RPS {'f_1': 0.3, 'f_2': 0.6, 'f_3': 0.6}
RPS {'f_1': 0.3, 'f_2': 0.2, 'f_3': 0.5}
RPS {'f_1': 0.4, 'f_2': 0.1, 'f_3': 0.4}
Final solution candidate: {'f_1': 0.36843260937649824, 'f_2': 0.8392004584885355, 'f_3': 0.4000000035258208}


# Ask if the DMs agree upon the final solution. Otherwise go to group discussion.

# Group discussion:
Goal is to help the DMs understand what is going on and what can they do about it, for example, why the found solution does not differ too much from the current one and how they should adjust their preferences if they are not willing to stop with this solution but find something different.

- give DMs information about the state; what is achievable and what is not, unless preferences are changed. explain why not moving etc.
- Nudge or ask DMs to adjust their preferences to find better suiting solutions for the group.
- Finally, set next_current_solution to gnimbus_solution and go back to DMs giving preferences for the next iteration

In [9]:
next_current_solution = gnimbus_solution